[[source]](../api/alibi.explainers.kernel_shap.rst)

# Tree SHAP

## Overview

The tree SHAP (**SH**apley **A**dditive ex**P**lanations) algorithm is based on the paper [From local explanations to global understanding with explainable AI for trees](https://www.nature.com/articles/s42256-019-0138-9) by Lundberg et al. and builds on the open source  [shap library](https://github.com/slundberg/shap) from the paper's first author.

### How it works


Recall that, for a model $f$, the Kernel SHAP algorithm [[1]](#References) explains a certain outcome with respect to a chosen reference (or an expected value) by estimating the shap values of each feature $i \in F = \{1, ..., M\} $, as follows:

- enumerate all subsets $S$ of the set $F   \setminus \{i\}$ 
- for each $S \subseteq F \setminus \{i\}$, compute the contribution of feature $i$ as $C(i|S) = f(S \cup \{i\}) - f(S)$
- compute the shap value according to
$$
\phi_i := \frac{1}{n} \sum \limits_{{S \subseteq F \setminus \{i\}}} \frac{1}{ n - 1 \choose |S|} C(i|S).         
\tag{1}
$$


Since most models do not accept arbitrary patterns of missing values at inference time, $f(S)$ is approximated using _interventional conditional expectations_ [[2]](#References). For Kernel SHAP, these expectations are estimated by replacing the _missing values_ ($\bar{S}$) with values from a _background dataset_ and taking an expectation over the model output. In the context of tree models, this approximation can be performed by: 
- traversing the tree with the example to be explained (sometimes referred to as a _foreground sample_) and a sample from the background dataset. This is algorithm is known as interventional feature perturbation
- without a background dataset, by exploiting the tree coverage information (i.e., information about how the training data was partitioned by the tree) along with the feature dependencies encoded in the tree paths
<a id='source_1'></a>

The following sections detail how these methods work and how, unlike Kernel SHAP, compute the exact shap values in polynomial time. 



#### Path dependent feature perturbation
<a id='path_dependent'></a>

A way to compute $f(S)$ given an instance $x$ and a set of missing features $\bar{S}$ is to recursively follow the decision path through the tree and:
- return the node value if a split on a feature $i \in S$ is performed
- take a weighted average of the values returned by children if $i \in \bar{S}$, where the weighing factor is equal to the proportion of training examples flowing down each branch. This proportion is a property of each node, sometimes referred to as _weight_ or _cover_. 

Therefore, in the path-dependent perturbation method, we compute the conditional expectations with respect to the training data distribution by weighting the leaf values according to the proportion of the training examples that flow to that leaf.

To avoid repeating the above recursion $M2^M$ times, one first notices that for a single decision tree, applying a perturbation would result in the sample ending up in a different leaf. Therefore, following each path from the root to a leaf in the tree is equivalent to perturbing subsets of features of varying cardinalities. Consequently, each leaf will contain a certain proportion of all possible subsets $S \subseteq F$. Therefore, to compute the Shap values, the following quantities are computed at each leaf, *for every feature $i$ on the path leading to that leaf*:
- the proportion of subsets $S$ at the leaf that contain $i$ and the proportion of subsets $S$ that do not contain $i$
- for each cardinality, the proportion of the sets of that cardinality contained at the leaf. Tracking each cardinality  as opposed to a single count of subsets falling into a given leaf is necessary since it allows to apply the weighting factor in equation (1), which depends on the subset size, $|S|$.

Using the above quantities, one can compute the _contribution_ of each leaf to the Shapley value of every feature. This algorithm has complexity $O(TLD^2)$ for an ensamble of trees where $L$ is the number of leaves, $T$ the number of trees in the ensamble and $D$ the maximum tree depth. If the tree is balanced, then $D=\log L$ and the complexity of our algorithm is $O(TL\log^2L)$

$$
\phi_i := \sum \limits_{j=1}^L \sum \limits_{P \in {S_t}} \frac {w(|P|, j)}{ M_j {M_j \choose |P|}} \left[f(P \cup \{i\}) - f(P)\right]
\tag{2}
$$

where $S_t$ is the set of present feature subsets at leaf $j$, $M_j$ is the length of the path and $w(|P|, j)$ is the proportion of all subsets of cardinality $P$ at leaf $j$.

- Include discussion about zero and one fractions and cover
- Include discussion about cover/Hessian 
- Make sure above is actually correct, add missing stuff to the equation and put it in context

Note that although a background dataset is not provided, the expected values is computed using the node cover information, stored at each node. The computation proceeds recursively, starting at the root. The contribution of a note to the expected value of the tree is a function of the expected values of the children and is computed as follows:

$$
c_j = \frac{c_{r(j)}r_{r(j)} + c_{l(j)}r_{l(j)}}{r_j}
$$

where $j$ denotes the node index, $c_j$ denotes the node expected value, $r_j$ is the cover of the $j$th node and $r(j)$ and $l(j)$ represent the indices of the right and left children, respectively. The expected value used by the tree is simply $c_{root}$. Note that for tree ensamles, the expected values of the ensamble members is weighted according to the tree weight and the weighted expected values of all trees are summed to obtain a single value.

The cover depends on the objective function and the model chosen. For example, in a gradient boosted tree trained with squared loss objective, $r_j$ is simply the number of training examples flowing through $j$. For an arbitrary objective, this is the sum of the hessian of the loss function evaluated at each point flowing through $j$, as explained [here](../../../examples/tree_shap_adult_xgb.ipynb#optimisation). 

#### Interventional feature perturbation
<a id='interventional'></a>


- Describe interventional perturbation concept and why it is a good idea - try and give intuition
- Summary of what the difference to the other algorithm is - aka how we estimate the shap values using this method
- Foreground sample and background sample concept
- State complexity
- Applying the interventional feature perturbation algorithm is equivalent to making an independence assumption between the set of features corresponding to the nodes where the foreground sample and the background samples are split in the same subtree and the set of features corresponding to nodes where 
- Explaining loss functions
- Other can only explain output, we can explain a nonlinear transformation on this one

###### Explaining loss functions

One advantage of the interventional approach is that it allows to approximately transform the shap values to account for nonlinear transformation of outputs, such as the loss function. Recall that given $\phi_i, ..., \phi_M$ the local accuracy property gurantees that given $\phi_0 = \mathbb{E}[f(x)]$

$$
f(x) = \phi_0 + \sum \limits_{i=1}^M \phi_i.
\tag{3}
$$

Hence, in order to account for the effect of the nonlinear transformation $h$, one has to find the functions $g_0, ..., g_M$ such that

$$
h(f(x)) = g(\phi_0) + \sum \limits_{i=1}^M g_i(\phi_i)
\tag{4}
$$

For simplicity, let $y=h(x)$. Then using a first-order Taylor series expansion around $\mathbb{E}[y]$ one obtains 

$$
f(y) \approx f(\mathbb{E}[y]) + \frac{\partial f(y) }{\partial y} \Bigr|_{y=\mathbb{E}[y]}(y - \mathbb{E}[y]).
\tag{5}
$$

Substituting $(3)$ in $(5)$ and comparing coefficients with $(4)$ yields

$$
g_0 \approx f(\mathbb{E}[y]) \\
g_i \approx \phi_i \frac{\partial f(y) }{\partial y} \Bigr|_{y=\mathbb{E}[y]} .
$$

Hence, an approximate correction is given by simply scaling the shap values using the gradient of the nonlinear function. Note that in practice one may take the Taylor series expasion at a reference point $r$ from the background dataset and average over the entire background dataset to compute the scaling factor. This introduces an additional source of noise since $f(\mathbb{E}[y]) = \mathbb{E}[f(y)]$ only for linear functions.

#### Shapley interaction values

- Describe Shapley interactions
- Describe the algorithm to compute them with reference to the path-dependent algorithm
- State computational complexity


### Comparison to other methods

- Feature importances (traditional)
    - inconsistency and arbitrary choice of explanation concept
    - global only, not that much info
- Kernel SHAP
    - post-hoc modelling so slow, but can be shown to converge 

## Usage

- summary of arguments to `explain` and what they mean

### Path-dependent perturbation algorithm

##### Initialiastion and Fit

##### Explanation

### Interventional perturbation algorithm

##### Explaining model output

##### Initialiastion and Fit

##### Explanation

#### Explaining loss functions

##### Initialisation and fit 

##### Explanation

### Shapley interaction values

##### Initialisation and fit 

#### Explanation

### Miscellaneous

#### Runtime considerations

##### Adjusting the size of the reference dataset

The algorithm automatically warns the user if a background dataset size of more than `300` samples is passed. If the runtime of an explanation with the original dataset is too large, then the algorithm can automatically subsample the background dataset during the `fit` step. This can be achieve by specifying the fit step as 

```python
explainer.fit(
    X_reference,
    summarise_background=True,
    n_background_samples=150,
)
```

or 
```python
explainer.fit(
    X_reference,
    summarise_background='auto'
)
```

The `auto` option will select `300` examples, whereas using the boolean argument allows the user to directly control the size of the reference set. If categorical variables or grouping options are specified, the algorithm uses subsampling of the data. Otherwise, a kmeans clustering algorithm is used to select the background dataset and the samples are weighted according to the frequency of occurence of the cluster they are assigned to, which is reflected in the `expected_value` attribute of the explainer. 

As describe above, the explanations are performed with respect to the expected (or weighted-average) output over this dataset so the shap values will be affected by the dataset selection. We recommend experimenting with various ways to choose the background dataset before deploying explanations.

##### The dimensionality of the data and the number of samples used in shap value estimation

## References

<a id='References'></a>

[[1]](#source_1) Lundberg, S.M. and Lee, S.I., 2017. A unified approach to interpreting model predictions. In Advances in neural information processing systems (pp. 4765-4774).

[[2]](#source_2) Janzing, D., Minorics, L. and Blöbaum, P., 2019. Feature relevance quantification in explainable AI: A causality problem. arXiv preprint arXiv:1910.13413.

## Examples

### Continuous Data

[Introductory example: Kernel SHAP on Wine dataset](../examples/kernel_shap_wine_intro.nblink)

[Comparison with interpretable models](../examples/kernel_shap_wine_lr.nblink)

### Mixed Data

[Handling categorical variables with Kernel SHAP: an income prediction application](../examples/kernel_shap_adult_lr.nblink)

[Handlling categorical variables with Kernel SHAP: fitting explainers on data before pre-processing](../examples/kernel_shap_adult_categorical_preproc.nblink)